In [92]:
import requests
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import folium
from time import time
import pickle

In [11]:
def get_live_station(url):
    
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    stations = response.json()
    
    output = []
    
    for station in stations['features']:
        dict_keys = ['kioskId', 'bikesAvailable', 'docksAvailable', 'name', 'latitude', 'longitude']
        data = {k : station['properties'][k] for k in dict_keys}
        data['time'] = time()
        output.append(data)

    return pd.DataFrame(output)

In [12]:
live_station_df = get_live_station("https://bikeshare.metro.net/stations/json/")
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,time
0,16,10,3005,34.04850,-118.25854,7th & Flower,1.569524e+09
1,10,14,3006,34.04554,-118.25667,Olive & 8th,1.569524e+09
2,8,15,3007,34.05048,-118.25459,5th & Grand,1.569524e+09
3,7,8,3008,34.04661,-118.26273,Figueroa & 9th,1.569524e+09
4,5,17,3010,34.03705,-118.25487,11th & Maple,1.569524e+09


In [13]:
# change time featur into str and take first 10 character.
live_station_df.time = live_station_df.time.astype(str)
live_station_df.time = live_station_df.time.str[:10]

In [14]:
# convert unix timestamp to Y/M/D %H/%M/%S format
live_station_df['time'] = pd.to_datetime(live_station_df['time'], unit='s')

In [15]:
# change name of time feature to ds and set it to index
# replace minutes and seconds to 0 
live_station_df.rename(columns={'time':'ds'}, inplace=True)
live_station_df.set_index('ds', inplace=True)
live_station_df.index = live_station_df.index.map(lambda x: x.replace(second=0))
live_station_df.index = live_station_df.index.map(lambda x: x.replace(minute=0))

In [16]:
live_station_df.kioskId = live_station_df.kioskId.astype(str)

In [26]:
live_station_df.head()

,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
ds,,,,,,
2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower
2019-09-26 18:00:00,10,14,3006,34.04554,-118.25667,Olive & 8th
2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand
2019-09-26 18:00:00,7,8,3008,34.04661,-118.26273,Figueroa & 9th
2019-09-26 18:00:00,5,17,3010,34.03705,-118.25487,11th & Maple


### Create new column concatenated with ds and kioskid to merge it with station_pred_df1

In [27]:
live_station_df.reset_index(inplace=True)

In [31]:
live_station_df['id'] = live_station_df.ds.astype(str) + '-' + live_station_df.kioskId
live_station_df.head()

,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,id
0,2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower,2019-09-26 18:00:00-3005
1,2019-09-26 18:00:00,10,14,3006,34.04554,-118.25667,Olive & 8th,2019-09-26 18:00:00-3006
2,2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand,2019-09-26 18:00:00-3007
3,2019-09-26 18:00:00,7,8,3008,34.04661,-118.26273,Figueroa & 9th,2019-09-26 18:00:00-3008
4,2019-09-26 18:00:00,5,17,3010,34.03705,-118.25487,11th & Maple,2019-09-26 18:00:00-3010


## Read forecast station df stores as in pickle

In [24]:
with open('station_pred_df1.pickle', 'rb') as f:
    station_pred_df1 = pickle.load(f)

station_pred_df1.head()

,station,ds,y_hat
0,3047.0,2019-09-24 13:00:00,0.439309
1,3047.0,2019-09-24 14:00:00,0.432759
2,3047.0,2019-09-24 15:00:00,0.434784
3,3047.0,2019-09-24 16:00:00,0.498354
4,3047.0,2019-09-24 17:00:00,0.608555


In [170]:
station_pred_df1.tail()

,station,y_hat,id
ds,,,
2019-10-06 18:00:00,3078,0.408277,2019-10-06 18:00:00-3078
2019-10-06 19:00:00,3078,0.371387,2019-10-06 19:00:00-3078
2019-10-06 20:00:00,3078,0.326630,2019-10-06 20:00:00-3078
2019-10-06 21:00:00,3078,0.282961,2019-10-06 21:00:00-3078
2019-10-06 22:00:00,3078,0.251276,2019-10-06 22:00:00-3078


In [25]:
# Take first 4 character in station feature
station_pred_df1.station = station_pred_df1.station.str[:4]

In [33]:
station_pred_df1['id'] = station_pred_df1.ds.astype(str) + '-' + station_pred_df1.station

In [37]:
station_pred_df1.tail()

,station,ds,y_hat,id
17995,3078,2019-10-06 18:00:00,0.408277,2019-10-06 18:00:00-3078
17996,3078,2019-10-06 19:00:00,0.371387,2019-10-06 19:00:00-3078
17997,3078,2019-10-06 20:00:00,0.326630,2019-10-06 20:00:00-3078
17998,3078,2019-10-06 21:00:00,0.282961,2019-10-06 21:00:00-3078
17999,3078,2019-10-06 22:00:00,0.251276,2019-10-06 22:00:00-3078


In [38]:
station_pred_df1.set_index('ds', inplace=True)

In [149]:
merge_live_pred_df = station_pred_df1.merge(live_station_df, on='id')
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.681051,2019-09-26 18:00:00-3047,2019-09-26 18:00:00,14,22,3047,34.03998,-118.26640,Pico & Flower
1,3005,4.124923,2019-09-26 18:00:00-3005,2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower
2,3023,0.586036,2019-09-26 18:00:00-3023,2019-09-26 18:00:00,10,12,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.659422,2019-09-26 18:00:00-3051,2019-09-26 18:00:00,6,10,3051,34.04542,-118.25352,7th & Broadway
4,3007,1.657867,2019-09-26 18:00:00-3007,2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand


In [150]:
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name
0,3047,0.681051,2019-09-26 18:00:00-3047,2019-09-26 18:00:00,14,22,3047,34.03998,-118.26640,Pico & Flower
1,3005,4.124923,2019-09-26 18:00:00-3005,2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower
2,3023,0.586036,2019-09-26 18:00:00-3023,2019-09-26 18:00:00,10,12,3023,34.05091,-118.24097,1st & Judge John Aiso
3,3051,0.659422,2019-09-26 18:00:00-3051,2019-09-26 18:00:00,6,10,3051,34.04542,-118.25352,7th & Broadway
4,3007,1.657867,2019-09-26 18:00:00-3007,2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand


In [151]:
def normalize_demand_number(record):
    if record['y_hat'] >= 4.5:
        return round(record['y_hat']) * 2.5
    elif record['y_hat'] >= 4:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 3:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 1:
        return round(record['y_hat']) * 3
    elif record['y_hat'] >= 0.7:
        return round(record['y_hat']) * 2
    elif record['y_hat'] >= 0.4:
        return round(record['y_hat']) * 1
    else:
        return 0

In [152]:
merge_live_pred_df['demand'] = merge_live_pred_df.apply(normalize_demand_number, axis=1)
merge_live_pred_df

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand
0,3047,0.681051,2019-09-26 18:00:00-3047,2019-09-26 18:00:00,14,22,3047,34.03998,-118.26640,Pico & Flower,1
1,3005,4.124923,2019-09-26 18:00:00-3005,2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower,8
2,3023,0.586036,2019-09-26 18:00:00-3023,2019-09-26 18:00:00,10,12,3023,34.05091,-118.24097,1st & Judge John Aiso,1
3,3051,0.659422,2019-09-26 18:00:00-3051,2019-09-26 18:00:00,6,10,3051,34.04542,-118.25352,7th & Broadway,1
4,3007,1.657867,2019-09-26 18:00:00-3007,2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand,6
5,3075,0.829424,2019-09-26 18:00:00-3075,2019-09-26 18:00:00,10,6,3075,34.04211,-118.25619,Broadway & 9th,2
6,3066,0.587705,2019-09-26 18:00:00-3066,2019-09-26 18:00:00,4,23,3066,34.06339,-118.23616,Spring & College,1
7,3082,0.982937,2019-09-26 18:00:00-3082,2019-09-26 18:00:00,5,20,3082,34.04652,-118.23741,Traction & Rose,2
8,3064,1.316154,2019-09-26 18:00:00-3064,2019-09-26 18:00:00,12,6,3064,34.04681,-118.25698,Grand & 7th,3
9,3052,0.652791,2019-09-26 18:00:00-3052,2019-09-26 18:00:00,15,19,3052,34.05110,-118.26456,7th & Bixel,1


In [159]:
def nexthour(record):
    if record['bikesAvailable'] - record['demand'] >= 0:
        return record['bikesAvailable'] - record['demand']
    else:
        return 0

In [160]:
merge_live_pred_df['nextHour'] = merge_live_pred_df.apply(nexthour, axis=1)
merge_live_pred_df.head()

,station,y_hat,id,ds,bikesAvailable,docksAvailable,kioskId,latitude,longitude,name,demand,nextHour
0,3047,0.681051,2019-09-26 18:00:00-3047,2019-09-26 18:00:00,14,22,3047,34.03998,-118.26640,Pico & Flower,1,13
1,3005,4.124923,2019-09-26 18:00:00-3005,2019-09-26 18:00:00,16,10,3005,34.04850,-118.25854,7th & Flower,8,8
2,3023,0.586036,2019-09-26 18:00:00-3023,2019-09-26 18:00:00,10,12,3023,34.05091,-118.24097,1st & Judge John Aiso,1,9
3,3051,0.659422,2019-09-26 18:00:00-3051,2019-09-26 18:00:00,6,10,3051,34.04542,-118.25352,7th & Broadway,1,5
4,3007,1.657867,2019-09-26 18:00:00-3007,2019-09-26 18:00:00,8,15,3007,34.05048,-118.25459,5th & Grand,6,2


In [174]:
merge_live_pred_df.station.unique()

array(['3047', '3005', '3023', '3051', '3007', '3075', '3066', '3082',
       '3064', '3052', '3029', '3006', '3018', '3035', '3032', '3030',
       '3074', '3022', '3069', '3042', '3026', '3054', '3024', '3038',
       '3037', '3062', '3068', '3058', '3067', '3040', '3036', '3056',
       '3008', '3031', '3034', '3063', '3077', '3048', '3049', '3016',
       '3014', '3011', '3076', '3033', '3027', '3019', '3046', '3010',
       '3020', '3025', '3028', '3065', '3057', '3045', '3081', '3078'],
      dtype=object)

In [ ]:
df = merge_live_pred_df.copy()
df.to_csv('df.csv')

In [161]:
def get_base_map(df):
    return folium.Map(location=[df.latitude.mean(),
                             df.longitude.mean()],
                   zoom_start=14,
                     tiles='OpenStreetMap')
base = get_base_map(merge_live_pred_df)
base

In [168]:
def add_station_markers(initial_map, df):
    #station location visualization
    
    out_map = initial_map
    for lat, lon, Name, Available, Opendocks, Next_hr_remaining, in zip(df['latitude'], df['longitude'], df['name'], 
                                df['bikesAvailable'], df['docksAvailable'], df['nextHour']):
        folium.Marker([lat,lon], popup=(str(Name).capitalize() + '<br>'
                                        '<br><b>Available: </b>' + str(Available) + '<br>'
                                        '<br><b>Opendocks: </b>' + str(Opendocks) + '<br>'
                                        '<br><b>Remaining Next Hour:</b> ' + str(Next_hr_remaining)),
                     icon=folium.Icon(color='green')).add_to(out_map)
    return out_map

markers = add_station_markers(base, merge_live_pred_df)
markers 

In [171]:
markers.save('markers.html')

In [172]:
!ls

Data                             capstone_TimeSeries.ipynb
LICENSE                          capstone_TimeSeries_hourly.ipynb
Live_station_status.ipynb        functions.py
README.md                        markers.html
TimeSeries_BikeStation.ipynb     station_pred_df.pickle
capstone_EDA.ipynb               station_pred_df1.pickle
capstone_TimeSeries-Copy1.ipynb


In [173]:
!pwd

/Users/flatiron/Flatiron/capstone_project/Bber
